# Section 3: Create quantum circuits

## TASK 3.3: Transpile and optimize circuits

1. Which of the following lists the preset transpiler pipeline stages in the correct order?

a. layout → routing → translation → optimization → scheduling → init<br>
b. init → layout → routing → translation → optimization → scheduling<br>
c. init → routing → layout → translation → scheduling → optimization<br>
d. layout → translation → routing → optimization → init → scheduling<br>


:::{dropdown}answer 
The answer is b. 

The preset pipeline in Qiskit’s staged pass managers is ordered as: **init → layout → routing → translation → optimization → scheduling**. 
The `init` stage handles early housekeeping (e.g., reducing multi-qubit ops to 1–2 qubits where needed), then **layout** selects a mapping from virtual to physical qubits, **routing** inserts SWAPs as required by hardware connectivity, **translation** compiles to the target basis gates, **optimization** reduces depth and cancels redundant operations, and **scheduling** (only when requested) inserts delays and accounts for timing.
:::

2. Which statement best describes the purpose of the **init** stage?

a. It always runs by default and performs hardware-aware scheduling.<br>
b. It performs SWAP insertion to ensure two-qubit connectivity.<br>
c. It primarily enables early, optional cleanups and translates gates acting on 3+ qubits into 1–2 qubit operations.<br>
d. It decomposes circuits into the backend basis and cancels adjacent CNOT gates.<br>


:::{dropdown}answer 
The answer is c. 

The **init** stage does very little by default, but it’s where you can insert early optimizations if desired. Since many downstream algorithms expect only single- and two-qubit operations, this stage also translates any operation acting on three or more qubits into 1–2 qubit gates.
:::

3. In the **layout** stage, which pass uses a subgraph isomorphism approach (VF2++) to find an ideal mapping of virtual to physical qubits?

a. SabreSwap<br>
b. VF2Layout<br>
c. DenseLayout<br>
d. TrivialLayout<br>


:::{dropdown}answer 
The answer is b. 

**VF2Layout** treats layout selection as a subgraph isomorphism problem and uses VF2++ to search for candidate layouts; if multiple candidates are found, a scoring heuristic picks the one with the lowest average error.
:::

4. Which statement about **TrivialLayout** in the layout stage is correct?

a. It is the only layout method used at all optimization levels.<br>
b. It randomly permutes qubits to maximize entangling gate parallelism.<br>
c. It maps qubits in circuit order and is used historically at optimization level 1 to try a perfect layout before other methods.<br>
d. It is the default stochastic router for SWAP insertion.<br>


:::{dropdown}answer 
The answer is c. 

**TrivialLayout** naively maps virtual qubits to identically numbered physical qubits and is (historically) tried at **optimization level 1** to see if a perfect layout exists; otherwise, other layout heuristics (such as **VF2Layout** or **SabreLayout**) are used.
:::

5. Why can repeated runs of the same circuit produce a **distribution** of depths after routing?

a. Because routing is solved exactly and deterministically for each run.<br>
b. Because the SabreSwap router is **stochastic**, so SWAP insertion can vary across runs.<br>
c. Because translation to basis gates happens before layout.<br>
d. Because scheduling always inserts the same delays.<br>


:::{dropdown}answer 
The answer is b. 

Finding an optimal SWAP mapping is NP-hard, so Qiskit uses the **stochastic** heuristic **SabreSwap**. Its randomness leads to different SWAP patterns on different runs, yielding a distribution of circuit depths and gate counts.
:::

6. In the **translation** stage, which of the following is true when the SWAP gate is **not** native to the backend?

a. It is executed directly as a hardware-native delay instruction.<br>
b. It is decomposed into **three CX** gates (plus any necessary single-qubit gates).<br>
c. It is replaced by a Toffoli (CCX) gate pair.<br>
d. It is removed because SWAPs are never required on hardware.<br>


:::{dropdown}answer 
The answer is b. 

When SWAP is not native to the target’s basis, it is decomposed into a sequence of native gates—commonly **three CNOT (CX)** gates—making SWAPs expensive on noisy hardware and motivating layout/routing that minimizes their use.
:::

7. Which preset **optimization level** includes passes such as **Collect2qBlocks**, **ConsolidateBlocks**, and **UnitarySynthesis**?

a. Level 0<br>
b. Level 1<br>
c. Level 2<br>
d. Level 3<br>


:::{dropdown}answer 
The answer is d. 

At **optimization level 3**, the staged pass manager adds deeper structural optimizations such as **Collect2qBlocks**, **ConsolidateBlocks**, and **UnitarySynthesis**, in addition to single-qubit optimizations and commutation-based cancellations.
:::

8. Which statement about the **scheduling** stage is correct?

a. Scheduling always runs by default after optimization.<br>
b. Scheduling must be explicitly requested (e.g., via `scheduling_method`), and it inserts **Delay** instructions to account for idle times.<br>
c. Scheduling is the stage that performs VF2 layout search.<br>
d. Scheduling translates to the target basis and cancels adjacent CNOTs.<br>


:::{dropdown}answer 
The answer is b. 

Scheduling **does not run by default**; you trigger it by specifying a `scheduling_method` (e.g., `"asap"`). It analyzes timing and inserts **Delay** instructions for idle periods, helping estimate execution timing on the backend.
:::

9. What is the **recommended** way to transpile a circuit in Qiskit?

a. Call `transpile(circuit)` only; pass managers are deprecated.<br>
b. Create a **preset staged pass manager** with `generate_preset_pass_manager(...)` and call `run(circuit)`.<br>
c. Directly construct a `StagedPassManager` is forbidden; only `PassManager` is allowed.<br>
d. Only schedule the circuit; layout and routing are optional and unnecessary.<br>


:::{dropdown}answer 
The answer is b. 

The recommended workflow is to build a **preset staged pass manager** with `generate_preset_pass_manager` (choosing an `optimization_level`, backend, and options) and then invoke its **`run`** method on the circuit.
:::

10. Which description best characterizes a **staged pass manager**?

a. A pass manager composed directly of gates instead of passes.<br>
b. A pass manager that contains one monolithic pass implementing all stages.<br>
c. An abstraction that is **composed of multiple pass managers**, each representing a transpiler stage (e.g., layout, routing, translation).<br>
d. A pass manager that only schedules circuits and cannot do layout or routing.<br>


:::{dropdown}answer 
The answer is c. 

A **StagedPassManager** is built from per-stage **PassManager** instances (e.g., for layout, routing, translation), providing a higher-level abstraction aligned with the logical stages of transpilation.
:::

11. You start from a preset staged pass manager at optimization level 3 and want to add extra **logical optimization before layout**. What is a supported approach?

a. You cannot modify a preset pass manager.<br>
b. Assign a custom `pre_layout` stage (e.g., an `InverseCancellation` pass list) to the staged pass manager.<br>
c. Replace the router with `generate_embed_passmanager`, which is strictly for scheduling.<br>
d. Insert scheduling analysis passes into the translation stage by default.<br>


:::{dropdown}answer 
The answer is b. 

You can **augment** a preset staged pass manager by adding custom stages like **`pre_layout`** (e.g., with `InverseCancellation`) or by swapping out stages. Stage generator helpers such as `generate_embed_passmanager` can assist when building custom stages.
:::

12. Which statement about `transpile` vs. preset staged pass managers is accurate?

a. `transpile` does not use pass managers internally.<br>
b. `transpile` internally **calls** `generate_preset_pass_manager` and runs it on the circuit.<br>
c. `transpile` always performs scheduling by default.<br>
d. `transpile` can only accept lists of circuits, not a single circuit.<br>


:::{dropdown}answer 
The answer is b. 

The simpler `transpile` function **internally** creates a preset staged pass manager via `generate_preset_pass_manager` using the given options (e.g., `optimization_level`) and then runs it on the input circuit(s).
:::
